Import dataset

In [1]:
import numpy as np
import torch
from data.dataset import OneDDatasetLoader, DatasetLoader
from typing import List
from preprocessing.batching import merge_graphs
from torch_geometric.loader import DataLoader

def train_eval_split(dataset : DatasetLoader, train_id : List, eval_id : List):
    # Get batching id
    if dataset._sub_dir == '/batched/':
        batching_id = dataset.batching_id.numpy()
        train_id = list(np.where(np.isin(batching_id, train_id) == True)[0])
        eval_id = list(np.where(np.isin(batching_id, eval_id) == True)[0])
    # Train dataset
    train_dataset = [dataset[i] for i in train_id]
    # Test dataset
    eval_dataset = [dataset[i] for i in eval_id]
    return train_dataset, eval_dataset

In [2]:
dataset = OneDDatasetLoader(
    root_dir='/data1/tam/downloaded_datasets_nodeattr_v3',
    sub_dir='/batched/'
)
print('Dataset loaded.')
train_dataset, test_dataset = train_eval_split(dataset,
    train_id=list(range(0,10)), eval_id=list(range(30,40)))

Dataset loaded.


In [3]:

train_loader = DataLoader(train_dataset, batch_size=5)
test_loader = DataLoader(test_dataset, batch_size=5)

Train

In [4]:
import os
import torch
from networks.network_parc_v4 import PARC_reducedV4 as PARC
# from networks.network_parc import PARC_reduced
os.environ["CUDA_VISIBLE_DEVICES"]="2"
from networks.network_recurrent import objectview
from networks.loss import WeightedMSELoss
CUDA_LAUNCH_BLOCKING=1
torch.cuda.empty_cache()

# Model params
args = objectview({
    'n_fields' : 2,
    'n_timesteps' : 1,
    'n_hiddenfields' : 128,
    'n_meshfields' : 10, # dataset[0].node_attr.size(1),
    'n_bcfields' : 1,
    'time' : 4.0,
    'device' : torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    'lr' : 2e-7,
    'weight_decay' : 5e-4,
    'epoch' : 100,
    'train_lambda' : 0.5
})

# # Model initializing

model = PARC(
    n_fields=args.n_fields,
    n_timesteps=args.n_timesteps,
    n_hiddenfields=args.n_hiddenfields,
    n_meshfields=args.n_meshfields,
    n_bcfields=args.n_bcfields
).to(args.device)

optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

criterion = torch.nn.MSELoss()
# criterion = WeightedMSELoss()

In [5]:
# Train function v2
def train(model, data, args):
    n_time = data.pressure.size(1)
    timesteps = args.time / n_time
    # edge_index = data.edge_index.to(args.device)
    edge_index = torch.cat([
        data.edge_index, torch.flip(data.edge_index, dims=[0]
        )], dim=1).to(args.device)
    mesh_features = data.node_attr.to(args.device)
    F_bc = data.flowrate_bc.to(args.device)
    # weight = data.node_weight.to(args.device)

    F_initial = torch.cat([
        data.pressure[:,0].unsqueeze(1), 
        data.flowrate[:,0].unsqueeze(1)
    ], dim=-1).to(args.device) # concat pressure and flowrate

    model.n_timesteps = n_time

    Fs, F_dots = model(F_initial, mesh_features, edge_index, F_bc, timesteps)

    Fs_hat = torch.cat([
        data.pressure.unsqueeze(-1), 
        data.flowrate.unsqueeze(-1)
    ], dim=-1).to(args.device) # concat pressure and flowrate
    
    # F_dots_hat = torch.cat([
    #     data.pressure_dot.unsqueeze(-1), 
    #     data.flowrate_dot.unsqueeze(-1)
    # ], dim=-1).to(args.device) # concat pressure and flowrate

    Fs_hat = Fs_hat[:,1:,:]
    # F_dots_hat = F_dots_hat[:,1:,:]
    
    # loss = (1.-args.train_lambda)*criterion(Fs_hat, Fs) + \
    #         args.train_lambda*criterion(F_dots_hat, F_dots)
    loss = criterion(Fs_hat, Fs)

    loss.backward()
    optimizer.step()
    return loss.item()

# Eval function
def eval(model, data, args):
    n_time = data.pressure.size(1)
    timesteps = args.time / n_time
    # edge_index = data.edge_index.to(args.device)
    edge_index = torch.cat([
        data.edge_index, torch.flip(data.edge_index, dims=[0]
        )], dim=1).to(args.device)
    mesh_features = data.node_attr.to(args.device)
    F_bc = data.flowrate_bc.to(args.device)
    # weight = data.node_weight.to(args.device)
    F_initial = torch.cat([
        data.pressure[:,0].unsqueeze(1), 
        data.flowrate[:,0].unsqueeze(1)
    ], dim=-1).to(args.device) # concat pressure and flowrate
    model.n_timesteps = n_time

    with torch.no_grad():
        Fs, F_dots = model(F_initial, mesh_features, edge_index, F_bc, timesteps)

        Fs_hat = torch.cat([
            data.pressure.unsqueeze(-1), 
            data.flowrate.unsqueeze(-1)
        ], dim=-1).to(args.device) # concat pressure and flowrate
        
        # F_dots_hat = torch.cat([
        #     data.pressure_dot.unsqueeze(-1), 
        #     data.flowrate_dot.unsqueeze(-1)
        # ], dim=-1).to(args.device)

        Fs_hat = Fs_hat[:,1:,:]
        # F_dots_hat = F_dots_hat[:,1:,:]

        # loss = (1.-args.train_lambda)*criterion(Fs_hat, Fs) + \
        #     args.train_lambda*criterion(F_dots_hat, F_dots)
        loss = criterion(Fs_hat, Fs)

    # return loss
    return loss.item()

# Training
total_train_loss = []
total_eval_loss = []
# batch = enumerate(list(range(0,10)))
for epoch in range(args.epoch):
    torch.cuda.empty_cache()
    train_loss = 0
    # for data in train_dataset:
    for i in range(train_loader.__len__()):
        data = next(iter(train_loader))
        train_loss += train(model=model, data=data, args=args)

    train_loss /= train_loader.__len__() # len(train_dataset)
    total_train_loss.append(train_loss)

    eval_loss = 0
    # for data in eval_dataset:
    for i in range(test_loader.__len__()):
        data = next(iter(test_loader))
        eval_loss += eval(model=model, data=data, args=args)
    eval_loss /= test_loader.__len__() #len(eval_dataset)
    total_eval_loss.append(eval_loss)
    
    # if (epoch > 25):
    #     args.train_lambda = 0.5
    print(f'Epoch {epoch}: train loss = {train_loss}; eval loss = {eval_loss}')
    if (epoch+1) % 25 == 0:
        torch.save(model.state_dict(), f'models/parc_v4-6_epoch{epoch+1}.pth')

Epoch 0: train loss = 0.4327315027108982; eval loss = 0.4554592967033386
Epoch 1: train loss = 0.44231866159529054; eval loss = 0.4629867075624206
Epoch 2: train loss = 0.44623501595711057; eval loss = 0.4625476896762848
Epoch 3: train loss = 0.44207632210519576; eval loss = 0.4561157518337858
Epoch 4: train loss = 0.43671658160301624; eval loss = 0.453199177980423
Epoch 5: train loss = 0.43671312915704036; eval loss = 0.4560881555080414
Epoch 6: train loss = 0.44168164132776; eval loss = 0.4618586599826813
Epoch 7: train loss = 0.4455535392466331; eval loss = 0.46248000860214233
Epoch 8: train loss = 0.4428548175583845; eval loss = 0.4578157067298889
Epoch 9: train loss = 0.4387847659347942; eval loss = 0.4555663764476776
Epoch 10: train loss = 0.43968207106400337; eval loss = 0.45923223922837453
Epoch 11: train loss = 0.44384601343602753; eval loss = 0.46231526136398315
Epoch 12: train loss = 0.44439467481597167; eval loss = 0.46062415838241577
Epoch 13: train loss = 0.44202553157536

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(total_train_loss)
plt.plot(total_eval_loss)
# plt.ylim(0,0.1)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(total_train_loss)
plt.plot(total_eval_loss)
# plt.ylim(0,0.1)
plt.show()

In [ ]:
args.device

Reconstruct CFD output

In [ ]:
# Model initializing
model = PARC_reduced(
    n_fields=args.n_fields,
    n_timesteps=args.n_timesteps,
    n_hiddenfields=args.n_hiddenfields,
    n_meshfields=args.n_meshfields,
    n_bcfields=args.n_bcfields
)
model = model.to(args.device)
model.load_state_dict(torch.load(
    'models/parc_v4-5_epoch50.pth',
    map_location={'cuda:1': 'cuda:0'}
))


In [ ]:
# Plot prediction/ground truth
import matplotlib.pyplot as plt

def plot_comparison(model, data):
    # prepare input tensors
    n_time = data.pressure.size(1)
    timesteps = args.time / n_time
    # edge_index = data.edge_index.to(args.device)
    edge_index = torch.cat([
        data.edge_index, torch.flip(data.edge_index, dims=[0]
        )], dim=1).to(args.device)
    mesh_features = data.node_attr.to(args.device)
    F_bc = data.flowrate_bc.to(args.device)
    weight = data.node_weight.to(args.device)
    F_initial = torch.cat([
        data.pressure[:,0].unsqueeze(1), 
        data.flowrate[:,0].unsqueeze(1)
    ], dim=-1).to(args.device) # concat pressure and flowrate
    model.n_timesteps = n_time - 1

    with torch.no_grad():
        Fs, F_dots = model(F_initial, mesh_features, edge_index, F_bc, timesteps)

        Fs_hat = torch.cat([
            data.pressure.unsqueeze(-1), 
            data.flowrate.unsqueeze(-1)
        ], dim=-1).to(args.device) # concat pressure and flowrate
        
        # F_dots_hat = torch.cat([
        #     data.pressure_dot.unsqueeze(-1), 
        #     data.flowrate_dot.unsqueeze(-1)
        # ], dim=-1).to(args.device)

        Fs_hat = Fs_hat[:,1:,:]
        # F_dots_hat = F_dots_hat[:,1:,:]

        # loss = (1.-args.train_lambda)*criterion(Fs_hat, Fs) + \
        #     args.train_lambda*criterion(F_dots_hat, F_dots)

    

    
    # plot
    for i_node in [200, 20000]:

        i_field = 1
        y_pred = Fs.cpu().numpy()[i_node,:,i_field]
        y_true = Fs_hat.cpu().numpy()[i_node,:,i_field]
        x = [i * 4.0 /200 for i in range(y_pred.shape[0])]
        # print(data.node_attr.numpy()[i_node, 6])
        plt.ylim(-1,1)
        plt.plot(x, y_pred, c='red', label='GNN Crank-Nicolson')
        plt.plot(x, y_true, c='blue', linestyle='dashdot', label='ground_truth')
        # plt.ylim([-1,1])
        plt.legend(loc='upper right')
        plt.ylabel('Flowrate', fontsize=20)
        plt.xlabel('Time', fontsize=20)
        plt.show()
    
    return 0

dataset = OneDDatasetLoader(
    root_dir='/data1/tam/downloaded_datasets_nodeattr_v3',
    sub_dir='/validation/'
)
print('Dataset loaded.')

plot_comparison(model, dataset[40])

In [ ]:
# Plot prediction/ground truth
import matplotlib.pyplot as plt

def plot_comparison(model, data):
    # prepare input tensors
    n_time = data.pressure.size(1)
    timesteps = args.time / n_time
    # edge_index = data.edge_index.to(args.device)
    edge_index = torch.cat([
        data.edge_index, torch.flip(data.edge_index, dims=[0]
        )], dim=1).to(args.device)
    mesh_features = data.node_attr.to(args.device)
    F_bc = data.flowrate_bc.to(args.device)
    weight = data.node_weight.to(args.device)
    F_initial = torch.cat([
        data.pressure[:,0].unsqueeze(1), 
        data.flowrate[:,0].unsqueeze(1)
    ], dim=-1).to(args.device) # concat pressure and flowrate
    model.n_timesteps = n_time - 1

    with torch.no_grad():
        Fs, F_dots = model(F_initial, mesh_features, edge_index, F_bc, timesteps)

        Fs_hat = torch.cat([
            data.pressure.unsqueeze(-1), 
            data.flowrate.unsqueeze(-1)
        ], dim=-1).to(args.device) # concat pressure and flowrate
        
        F_dots_hat = torch.cat([
            data.pressure_dot.unsqueeze(-1), 
            data.flowrate_dot.unsqueeze(-1)
        ], dim=-1).to(args.device)

        Fs_hat = Fs_hat[:,1:,:]
        F_dots_hat = F_dots_hat[:,1:,:]

        # loss = (1.-args.train_lambda)*criterion(Fs_hat, Fs) + \
        #     args.train_lambda*criterion(F_dots_hat, F_dots)

    

    
    # plot
    for i_node in [1, 100, 1000, 10000, 50000]:

        i_field = 1
        y_pred = Fs.cpu().numpy()[i_node,:,i_field]
        y_true = Fs_hat.cpu().numpy()[i_node,:,i_field]
        x = [i * 4.0 /200 for i in range(y_pred.shape[0])]
        print(data.node_attr.numpy()[i_node, 6])
        plt.ylim(-1,1)
        plt.plot(x, y_pred, c='red', label='PIGNN')
        plt.plot(x, y_true, c='blue', linestyle='dashdot', label='ground_truth')
        # plt.ylim([-1,1])
        plt.legend(loc='upper right')
        plt.ylabel('flowrate', fontsize=20)
        plt.xlabel('Time', fontsize=20)
        plt.show()
    
    return 0

dataset = OneDDatasetLoader(
    root_dir='/data1/tam/downloaded_datasets_nodeattr_v3',
    sub_dir='/validation/'
)
print('Dataset loaded.')

plot_comparison(model, dataset[40])